In [37]:
import numpy as np
import pandas as pd
import os
from glob import glob
import re

In [38]:
eps=1e-16

In [39]:
# train_epa = np.concatenate([np.load('./epa/epa%s.npy'%(y)) for y in range(2014,2018+1)],axis=0)
# np.save('./epa/train_epa',train_epa)
# train_epa.shape

In [40]:
# test_epa = np.concatenate([np.load('./epa/epa%s.npy'%(y)) for y in range(2019,2019+1)],axis=0)
# np.save('./epa/test_epa',test_epa)
# test_epa.shape

# add zero

## epa

In [41]:
# train_epa = np.load('./epa/train_epa.npy')
# test_epa = np.load('./epa/test_epa.npy')

# train_zeros = np.zeros((43824, 4, 19))
# test_zeros = np.zeros((8760, 4, 19))

# train_epa_addmou = np.concatenate([train_zeros, train_epa],axis=1)
# test_epa_addmou = np.concatenate([test_zeros, test_epa],axis=1)

# np.save('./epa/train_epa_addmou.npy',train_epa_addmou)
# np.save('./epa/test_epa_addmou.npy',test_epa_addmou)

# train_epa_addmou.shape, test_epa_addmou.shape

In [42]:
# train_epa_addmou = np.load('./epa/train_epa_addmou.npy')
# test_epa_addmou = np.load('./epa/test_epa_addmou.npy')

# train_epa_addmou.shape, test_epa_addmou.shape

## sat, air

In [43]:
# path='/media/disk3/feynman52/station2grid/datasets/npy/air/domain_air-k_3-weightKNN_distance/station'
# station_paths=sorted(glob(os.path.join(path,'*_station.npy')))  
# for path in station_paths[:]:
#     arr = np.load(path)
#     zeros = np.zeros((1, 4, 3)) 
#     arr_addmou = np.concatenate([zeros, arr],axis=1)
#     path = path.replace('_station','_station_addmou')
#     np.save(path,arr_addmou)

# normalize

In [44]:
def normalize(A, min_, max_):
    eps = 1e-16
#     A_norm = (A-min_)/(max_-min_+eps) ### minmax
    A_norm = (A-min_)/(max_+eps) ### standard
    return A_norm

## normalize feature except pm25

In [45]:
train_epa_addmou = np.load('./epa/train_epa_addmou.npy')
test_epa_addmou = np.load('./epa/test_epa_addmou.npy')

train_epa_addmou.shape, test_epa_addmou.shape

((43824, 77, 19), (8760, 77, 19))

In [46]:
B_feature = np.mean(train_epa_addmou, axis=0) #np.mean(train_epa_addmou, axis=0), np.min(train_epa_addmou, axis=0)
C_feature = np.std(train_epa_addmou, axis=0) #np.std(train_epa_addmou, axis=0), np.max(train_epa_addmou, axis=0)

B_feature.shape, C_feature.shape

((77, 19), (77, 19))

In [47]:
train_epa_addmou_norm = normalize(train_epa_addmou, B_feature, C_feature)
train_epa_addmou_norm.shape
train_epa_addmou_norm[:,10,12].mean(), train_epa_addmou_norm[:,10,12].std()
# train_epa_addmou_norm[:,0,12].mean(), train_epa_addmou_norm[:,0,12].std()


(1.90411963507818e-15, 1.000000000000017)

In [48]:
test_epa_addmou_norm = normalize(test_epa_addmou, B_feature, C_feature)
test_epa_addmou_norm.shape
test_epa_addmou_norm[:,10,12].mean(), test_epa_addmou_norm[:,10,12].std()
# test_epa_addmou_norm[:,0,12].mean(), test_epa_addmou_norm[:,0,12].std()


(-0.3196607180638597, 0.5848428796139685)

In [49]:
# np.save('./epa/train_epa_addmou_norm.npy',train_epa_addmou_norm)
# np.save('./epa/test_epa_addmou_norm.npy',test_epa_addmou_norm)


## normalize pm25

### train_mean, train_std

In [50]:
train_epa_addmou_pm25 = train_epa_addmou[:,:,0]
train_epa_addmou_pm25.shape

(43824, 77)

In [51]:
path='/media/disk3/feynman52/station2grid/datasets/npy/air/domain_air-k_3-weightKNN_distance/station'
station_paths=sorted(glob(os.path.join(path,'*_station_addmou.npy')))  
air_pm25 = [np.load(path)[:,:,0] for path in station_paths[:]]
air_pm25 = np.concatenate(air_pm25, axis=0)
air_pm25.shape

(30005, 77)

In [52]:
path='/media/disk3/feynman52/station2grid/datasets/npy/sat/domain_sat-k_3-weightKNN_distance/station'
station_paths=sorted(glob(os.path.join(path,'*_station_addmou.npy')))  
sat_pm25 = [np.load(path)[:,:,0] for path in station_paths[:]]
sat_pm25 = np.concatenate(sat_pm25, axis=0)
sat_pm25.shape

(4383, 77)

In [53]:
n_epa, n_air, n_sat = len(train_epa_addmou_pm25), len(air_pm25), len(sat_pm25)

In [54]:
train_pm25 = np.concatenate([train_epa_addmou_pm25, air_pm25, sat_pm25], axis=0)
train_pm25.shape

(78212, 77)

In [55]:
B_pm25 = np.mean(train_pm25,axis=0) #np.mean(train_pm25,axis=0), 0, np.min(train_pm25,axis=0)
C_pm25 = np.std(train_pm25,axis=0) #np.std(train_pm25,axis=0), 100, np.max(train_pm25,axis=0)

# B_pm25.shape, C_pm25.shape

### norm train pm25

In [56]:
# import matplotlib.pyplot as plt 
# plt.hist(B_pm25)

In [57]:
train_pm25_norm = normalize(train_pm25, B_pm25, C_pm25)
train_pm25_norm.shape
# i=10
# train_pm25_norm[:,i].mean(), train_pm25_norm[:,i].std()

(78212, 77)

### norm test pm25

In [58]:
test_pm25 = test_epa_addmou[:,:,0]
test_pm25.shape

(8760, 77)

In [59]:
test_pm25_norm = normalize(test_pm25, B_pm25, C_pm25)
test_pm25_norm.shape
i=5
test_pm25_norm[:,i].mean(), test_pm25_norm[:,i].std()

(-0.5485307892870324, 0.564510744581396)

### norm epa pm25

#### train

In [60]:
train_epa_addmou_norm[:,:,0].shape

(43824, 77)

In [61]:
train_pm25_norm[:n_epa].shape

(43824, 77)

In [62]:
train_epa_addmou_norm[:,:,0] = train_pm25_norm[:n_epa]

In [63]:
np.save('./epa/train_epa_addmou_standard_norm.npy',train_epa_addmou_norm)

#### test

In [64]:
test_epa_addmou_norm[:,:,0].shape

(8760, 77)

In [65]:
test_pm25_norm.shape

(8760, 77)

In [66]:
test_epa_addmou_norm[:,:,0] = test_pm25_norm

In [67]:
np.save('./epa/test_epa_addmou_standard_norm.npy',test_epa_addmou_norm)


### norm air pm25

In [68]:
import re
def station_path_to_dt(station_path):
    pat='station/(.+)_station'
    tar=station_path
    dt_str=re.search(pat, tar).group(1)
    dt_str_=dt_str[:-6] # process half hour
    return pd.to_datetime(dt_str_) 

In [69]:
all_epa = np.concatenate([train_epa_addmou,test_epa_addmou],axis=0)
all_epa_norm = np.concatenate([train_epa_addmou_norm,test_epa_addmou_norm],axis=0)
epa_dts = pd.date_range(start='2014',end='2020',closed='left',freq='H').tolist()

all_epa.shape, all_epa_norm.shape, len(epa_dts)

((52584, 77, 19), (52584, 77, 19), 52584)

In [70]:
air_pm25_norm = train_pm25_norm[n_epa : n_epa+n_air]
sat_pm25_norm = train_pm25_norm[n_epa+n_air : n_epa+n_air+n_sat]

air_pm25_norm.shape, sat_pm25_norm.shape

((30005, 77), (4383, 77))

In [71]:
path='/media/disk3/feynman52/station2grid/datasets/npy/air/domain_air-k_3-weightKNN_distance/station'
station_paths=sorted(glob(os.path.join(path,'*_station_addmou.npy')))  
for i,station_path in enumerate(station_paths[:]):
    domain_arr = np.load(station_path)
    
    dt = station_path_to_dt(station_path)
    i_dt = epa_dts.index(dt)
    epa_arr = all_epa[i_dt:i_dt+1]
    epa_arr_norm = all_epa_norm[i_dt:i_dt+1]
    
    domain_arr_addfea = np.concatenate([domain_arr,epa_arr], axis=-1)
    path = station_path.replace('_station_addmou','_station_addmou_addfea')
    np.save(path,domain_arr_addfea)
    
    domain_arr_addfea_norm = np.concatenate([domain_arr,epa_arr_norm], axis=-1)
    path = station_path.replace('_station_addmou','_station_addmou_addfea_standard_norm')
    domain_arr_addfea_norm[0,:,0] = air_pm25_norm[i]
    np.save(path,domain_arr_addfea_norm)

### norm sat pm25

In [72]:
path='/media/disk3/feynman52/station2grid/datasets/npy/sat/domain_sat-k_3-weightKNN_distance/station'
station_paths=sorted(glob(os.path.join(path,'*_station_addmou.npy')))  
for i,station_path in enumerate(station_paths[:]):
    domain_arr = np.load(station_path)
    
    dt = station_path_to_dt(station_path)
    i_dt = epa_dts.index(dt)
    epa_arr = all_epa[i_dt:i_dt+1]
    epa_arr_norm = all_epa_norm[i_dt:i_dt+1]
    
    domain_arr_addfea = np.concatenate([domain_arr,epa_arr], axis=-1)
    path = station_path.replace('_station_addmou','_station_addmou_addfea')
    np.save(path,domain_arr_addfea)
    
    domain_arr_addfea_norm = np.concatenate([domain_arr,epa_arr_norm], axis=-1)
    path = station_path.replace('_station_addmou','_station_addmou_addfea_standard_norm')
    domain_arr_addfea_norm[0,:,0] = sat_pm25_norm[i] ###
    np.save(path,domain_arr_addfea_norm)
#     print(station_path, domain_arr_addfea.shape, domain_arr_addfea_norm.shape)

# check

In [37]:
domain='sat'
path='/media/disk3/feynman52/station2grid/datasets/npy/%s/domain_%s-k_3-weightKNN_distance/station/2018-01-23 11:00:00_station.npy'%(domain,domain)
station = np.load(path)

path='/media/disk3/feynman52/station2grid/datasets/npy/%s/domain_%s-k_3-weightKNN_distance/station/2018-01-23 11:00:00_station_addmou.npy'%(domain,domain)
addmou = np.load(path)

path='/media/disk3/feynman52/station2grid/datasets/npy/%s/domain_%s-k_3-weightKNN_distance/station/2018-01-23 11:00:00_station_addmou_addfea.npy'%(domain,domain)
addmou_addfea = np.load(path)

path='/media/disk3/feynman52/station2grid/datasets/npy/%s/domain_%s-k_3-weightKNN_distance/station/2018-01-23 11:00:00_station_addmou_addfea_norm.npy'%(domain,domain)
addmou_addfea_norm = np.load(path)

station.shape, addmou.shape, addmou_addfea.shape, addmou_addfea_norm.shape

((1, 73, 3), (1, 77, 3), (1, 77, 22), (1, 77, 22))

In [38]:
i=1 # mountain
j=0 # pm25

print(addmou[0,i,0], addmou_addfea[0,i,j], addmou_addfea_norm[0,i,j])


0.0 0.0 0.0


In [39]:
# pm25

i=30 # city
j=0 # pm25

print(station[0,i-4,0], addmou[0,i,0], addmou_addfea[0,i,j], addmou_addfea_norm[0,i,j])

# print((addmou_addfea[0,i,0]-B_pm25[i])/(C_pm25[i]-B_pm25[i])) # minmax
# print(addmou_addfea[0,i,0]-B_pm25)/(C_pm25) # /100
print( (addmou_addfea[0,i,0]-B_pm25[i]) / (C_pm25[i]+eps) ) # standard



34.74532089637711 34.74532089637711 34.74532089637711 0.2003740186778056
0.20361676575120477


In [40]:
# non pm25

i=20 # station
j=20 # non pm25
print(station[0,i-4,0], addmou[0,i,0], addmou_addfea[0,i,j], addmou_addfea_norm[0,i,j])

# print(( addmou_addfea[0,i,j]-B_feature[i,j-3] ) / (C_feature[i,j-3]-B_feature[i,j-3])) # minmax
print( ( addmou_addfea[0,i,j]-B_feature[i,j-3] ) / (C_feature[i,j-3]+eps) ) # standard


17.21865990300766 17.21865990300766 1.8159619989581872 0.1377329364371591
0.1539321011720283


In [41]:
path='/media/disk3/feynman52/station2grid/datasets/npy/epa/train_epa_addmou_norm.npy'
train_epa_addmou_norm = np.load(path)

path='/media/disk3/feynman52/station2grid/datasets/npy/epa/train_epa_addmou.npy'
train_epa_addmou = np.load(path)

print(train_epa_addmou_norm.shape, train_epa_addmou.shape)
print(train_epa_addmou_norm[0,10,:], train_epa_addmou[0,10,:])

(43824, 77, 19) (43824, 77, 19)
[0.24103223 0.29530201 0.3255132  0.64634146 0.21469575 0.20205633
 0.51807229 0.28529121 0.02852486 0.         0.         0.
 0.15064935 0.19148936 0.16352201 0.27777778 0.         0.48964965
 0.47868614] [ 45.          89.          16.          71.           1.95
  90.          58.         148.           2.8          0.
   0.           0.           5.3          2.4          0.77
   3.1          0.           0.3146602   -0.38857298]


In [42]:
path='/media/disk3/feynman52/station2grid/datasets/npy/epa/test_epa_addmou_norm.npy'
test_epa_addmou_norm = np.load(path)

path='/media/disk3/feynman52/station2grid/datasets/npy/epa/test_epa_addmou.npy'
test_epa_addmou = np.load(path)

print(test_epa_addmou_norm.shape, test_epa_addmou.shape)
print(test_epa_addmou_norm[0,10,:], test_epa_addmou[0,10,:])

(8760, 77, 19) (8760, 77, 19)
[0.00535627 0.02684564 0.34897361 0.81707317 0.0619977  0.04336165
 0.14735867 0.06614018 0.21760391 0.         0.         0.
 0.08051948 0.06382979 0.03563941 0.07407407 0.         0.39790994
 0.64266036] [ 1.          9.         16.8        85.          0.62       19.
 18.         37.         26.          0.          0.          0.
  2.6         1.8         0.16        2.          0.         -0.71984337
  3.11798421]
